# Содание и анализ исторических доходностей индексов криптовалют

## 1.Введение

Попробуем создать простейший капитализационно-взвешенный индекс криптовалют по аналогии с индексами акций: S&P500, FTSE100, IMOEX и т.д. Созданный индекс будем использовать в качестве бенчмарка, т.е. образца для сравнения. Поскольку бенчмарк включает в себя самые крупные по капитализации криптовалюты, то можно считать, что его поведение близк*о* к поведению рынка криптовалют в целом.

Затем попробуем "поиграть" весами отдельных криптовалют (далее - тикеров) таким образом, чтобы среднегодовая доходность модифицированного индекса превысила доходность бенчмарка за тот же период.

<font color='red'>Основная гипотеза</font> состоит в том, что если при определении весов в индексе учитывать не только капитализацию каждого тикера, но и некую стадию жизненного цикла соответствующего криптопроекта, то удастся получить индекс, обыгрывающий бенчмарк. Текущую фазу проекта можно определять через:
- время вхождения в индекс;
- скорость роста капитализации;
- кратность роста капитализации со старта вхождения в индекс.

## 2. Источник данных

В качестве источника исторических данных возьмём самый известный сайт с информацией по данной теме: [coinmaketcap.com](https://coinmarketcap.com/)

![alt text](images/CMC_Screenshot.png)

К сожалению, необходимые нам исторические данные в бесплатном API не доступны.

![alt text](images/CMC_API_prices.png)

Включив режим "бедного студента" и поизучав подробнее сайт, мы видим, что в разделе по адресу [https://coinmarketcap.com/historical/](https://coinmarketcap.com/historical/) совершенно бесплатно доступны снэпшоты (снимки) исторических данных по ценам и капитализации всех криптовалют с периодичностью в одну неделю (каждое воскресенье) с 2013 года по текущий момент, то есть за последние 12 лет!

![alt text](images/CMC_HDS.png)

Так, например по адресу [https://coinmarketcap.com/historical/20130428/](https://coinmarketcap.com/historical/20130428/) можно увидеть первый доступный исторический снэпшот из далекого 28 апреля 2013 года, когда на сайте официально числилось лишь 7 криптовалют.

![alt text](images/CMC_H_20130428.png)

Казалось бы, кладезь полезной информации и совершенно бесплатно. Зачем тогда нужен API, тем более платный? Но как всегда в таких случаях есть нюанс: скачать все данные за раз в формате CSV, JSON, XML и т.п. нельзя. Нельзя скачать данные ни за всё время, ни за год, на даже за одну неделю.


## 3. Парсинг данных

### 3.1. План обработки данных

В связи невозможностью скачивания искомых даных похоже придётся прибегнуть к веб-скрейпингу. Для этого напишем скрипт на Python, который будет открывать все доступные страницы с историческими данными, и выгружать полезные данные на диск, где позже их можно будет объединить в общий Pandas DataFrame.

План действий следующий:
- Рассчитаем необходимое число снапшотов для обработки
- Определим необходимый набор столбцов
- Попробуем считать данные с одной страницы (снэпшота)
- Визуально проверим однотипность данных в начале и конце 12-летнего периода
- Попробуем считать данные для нескольких снэпшотов за раз
- Создадим и запустим скрипт для пакетного считывания данных
- Объединим данные в Pandas DataFrame

### 3.2. Расчет числа снэпшотов

In [1]:
# Импортируем pandas и функции из datetime
import pandas as pd
from datetime import datetime, timedelta

# Начальное число снэпшотов = 0
snapshots_number = 0

# Начальная дата = 28.04.2013
start_date = pd.to_datetime("20130428", format="%Y%m%d")

while True:
    potential_end_date = start_date + timedelta(weeks = snapshots_number)
    if potential_end_date >= datetime.now():
        break
    snapshots_number += 1
end_date = potential_end_date + timedelta(weeks = -1)
print(f"Всего снэпшотов для обработки: {snapshots_number}, последний датируется: {end_date.date()}")

Всего снэпшотов для обработки: 622, последний датируется: 2025-03-23


Скрипт запускался 25.01.2025, поэтому получилось всего 613 снэпшотов, последний из которых датировался 19.01.2025.

### 3.3. Необходимый набор столбцов

Здесь всё просто, возьмём для начала все присутствующие на первом снэпшоте столбцы:

| № | Название | Описание |
| --- | --- | --- |
| 01 | Rank | Порядковый номер криптовалюты в списке. Сортировка по капитализации по убыванию |
| 02 | Name | Название |
| 03 | Symbol | Тикер | 
| 04 | MarketCap | Капитализация |
| 05 | Price | Цена |
| 06 | Cirulating Supply | Количество монет в обращении |
| 07 | % 1h| Изменение цены за час |
| 08 | % 24h| Изменение цены за сутки |
| 09 | % 7d| Изменение цены за неделю |

### 3.4. Считывание данных с одной страницы

Попробуем считать данные со странички певого доступного снэпшота от 28.04.2013 и сохранить их в JSON-файл с именем соответственно "20130428.json". В коде первый снэпшот будет иметь номер #0.

In [83]:
# Импорт нужных библиотек
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import random
import os
import json

# Строка с датой исторически первого доступного снэпшота
first_snapshot_date_string = "20130428"

# Номер начального снэпшота для записи в JSON-файлы
start_snapshot_number = 0
# Номер конечного снэпшота для записи в JSON-файлы
stop_snapshot_number = 0

for current_snapshot_number in range(start_snapshot_number, stop_snapshot_number + 1):
    # Указываем актуальный путь к chromedriver
    service = ChromeService(executable_path='/usr/bin/chromedriver')
    options = Options()
    options.headless = True  # Запуск в фоновом режиме

    try:
        # Переход на страницу
        driver = webdriver.Chrome(service=service, options=options)
        current_snapshot_date_string = str(pd.to_datetime(first_snapshot_date_string) \
            + timedelta(weeks=current_snapshot_number)).split(' ')[0].replace("-","")
        driver.get("https://coinmarketcap.com/historical/" + current_snapshot_date_string +"/")
        time.sleep(15)  # Дополнительная задержка для полной загрузки данных

        # Получаем HTML-код страницы
        page_source = driver.page_source

    finally:
        driver.quit()

    # Парсинг HTML
    soup = BeautifulSoup(page_source, 'html.parser')

    columns_number = 10

    # Извлечение заголовков таблицы (почему-то нужно брать только первые 10 заголовков, далее идут повторы до 22 заголовков)
    headers = [th.text.strip() for th in soup.find_all('th')][:columns_number]

    # Извлечение данных из строк таблицы
    rows = []
    for row in soup.find_all('tr')[1:21]:  # Пропускаем первую строку с заголовками
        cells = row.find_all('td')
        if len(cells) > 0:
            row_data = [cell.text.strip() for cell in cells]
            rows.append(row_data)
    # Создание DataFrame
    temp_df = pd.DataFrame(rows, columns=headers)

    # Сохраняем файл
    json_filename = f"{current_snapshot_date_string}.json"
    temp_df.to_json(json_filename, orient='records', lines=True, force_ascii=False)
    print(f"{str(datetime.now()).split('.')[0]} Обаботан снэпшот № {current_snapshot_number + 1}.")
    

2025-02-12 01:09:34 Обаботан снэпшот № 1, делаю паузу 22 сек.


Скрипт отработал без ошибок. На считывание одного снэпшота ушло порядка 30 секунд. Теперь выведем содержимое получившегося JSON-файла:

In [82]:
!cat 20130428.json

{"Rank":"1","Name":"BTCBitcoin","Symbol":"BTC","Market Cap":"$1,488,566,971.96","Price":"$134.21","Circulating Supply":"11,091,325 BTC","% 1h":"0.64%","% 24h":"--","% 7d":"--","":""}
{"Rank":"2","Name":"LTCLitecoin","Symbol":"LTC","Market Cap":"$74,637,021.57","Price":"$4.3484","Circulating Supply":"17,164,230 LTC","% 1h":"0.80%","% 24h":"--","% 7d":"--","":""}
{"Rank":"3","Name":"PPCPeercoin","Symbol":"PPC","Market Cap":"$7,250,186.65","Price":"$0.3865","Circulating Supply":"18,757,362 PPC","% 1h":"-0.93%","% 24h":"--","% 7d":"--","":""}
{"Rank":"4","Name":"NMCNamecoin","Symbol":"NMC","Market Cap":"$5,995,997.19","Price":"$1.1072","Circulating Supply":"5,415,300 NMC","% 1h":"-0.05%","% 24h":"--","% 7d":"--","":""}
{"Rank":"5","Name":"TRCTerracoin","Symbol":"TRC","Market Cap":"$1,503,099.40","Price":"$0.6469","Circulating Supply":"2,323,570 TRC","% 1h":"0.61%","% 24h":"--","% 7d":"--","":""}
{"Rank":"6","Name":"DVCDevcoin","Symbol":"DVC","Market Cap":"$1,424,087.30","Price":"$0.0003261

А так выглядит датафрейм первого снэпшота:

In [66]:
temp_df

,Rank,Name,Symbol,Market Cap,Price,Circulating Supply,% 1h,% 24h,% 7d,
0,1,BTCBitcoin,BTC,"$1,488,566,971.96",$134.21,"11,091,325 BTC",0.64%,--,--,
1,2,LTCLitecoin,LTC,"$74,637,021.57",$4.3484,"17,164,230 LTC",0.80%,--,--,
2,3,PPCPeercoin,PPC,"$7,250,186.65",$0.3865,"18,757,362 PPC",-0.93%,--,--,
3,4,NMCNamecoin,NMC,"$5,995,997.19",$1.1072,"5,415,300 NMC",-0.05%,--,--,
4,5,TRCTerracoin,TRC,"$1,503,099.40",$0.6469,"2,323,570 TRC",0.61%,--,--,
5,6,DVCDevcoin,DVC,"$1,424,087.30",$0.0003261,"4,366,620,160 DVC",0.46%,--,--,
6,7,NVCNovacoin,NVC,"$1,162,266.30",$4.2464,"273,706 NVC",2.14%,--,--,


Отличный результат! С такой таблицей уже можно работать, несмотря на некоторые "корявости", например названия типа "BTCBitcoin".

### 3.5. Проверка однотипности данных

Если сравнить первые 34 снэпшота (по 22.12.2013 включительно) с любыми последующими, то можно заметить, что начиная с 35-го снэпшота в таблице добавился новый, седьмой столбец "volume (24h)". Теперь список столбцов выглядит следующим образом:

| № | Название | Описание |
| --- | --- | --- |
| 01 | Rank | Порядковый номер криптовалюты в списке. Сортировка по капитализации по убыванию |
| 02 | Name | Название |
| 03 | Symbol | Тикер | 
| 04 | MarketCap | Капитализация |
| 05 | Price | Цена |
| 06 | Cirulating Supply | Количество монет в обращении |
| <font color='red'>07</font> | <font color='red'>volume (24h)</font> | <font color='red'>Объём торгов за сутки</font> |
| 08 | % 1h| Изменение цены за час |
| 09 | % 24h| Изменение цены за сутки |
| 10 | % 7d| Изменение цены за неделю |

Следовательно, во избежании ошибок и путанницы при переходе к групповой загрузке данных и, тем более, пакетной загрузке, необходимо учесть данную особенность. Предлагается для первых 34 снэпшотов заполнить седьмой столбец нулями.

### 3.6. Групповая загрузка данных

Для демонстрации групповой загрузки во избежание лишней траты времени возьмём минимально возможную группу снэпшотов, состояющую всего из двух снэпшотов: #34 (22.12.2013) и #35 (29.12.2013). Номера выбраны специально, чтобы проверить корректность работы скрипта как со снэпшотом с 9 столбцами, так и с 10 столбцами (см. п. 3.5). В коде скрипта при желании всегда можно поменять начальный и конечный номер.

In [85]:
# Импорт нужных библиотек
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import random
import os
import json

# Номер начального снэпшота для записи в JSON-файлы
start_snapshot_number = 34
# Номер конечного снэпшота для записи в JSON-файлы
stop_snapshot_number = 35

# Строка с датой исторически первого доступного снэпшота, где начали указывать колонку volume 24h
first_snapshot_with_volume = 35
# Строка с датой исторически первого доступного снэпшота
first_snapshot_date_string = "20130428"

for current_snapshot_number in range(start_snapshot_number, stop_snapshot_number + 1):
    # Указываем актуальный путь к chromedriver
    service = ChromeService(executable_path='/usr/bin/chromedriver')
    options = Options()
    options.headless = True  # Запуск в фоновом режиме

    try:
        # Переход на страницу
        driver = webdriver.Chrome(service=service, options=options)
        current_snapshot_date_string = str(pd.to_datetime(first_snapshot_date_string) \
            + timedelta(weeks=current_snapshot_number)).split(' ')[0].replace("-","")
        driver.get("https://coinmarketcap.com/historical/" + current_snapshot_date_string +"/")
        time.sleep(15)  # Дополнительная задержка для полной загрузки данных

        # Получаем HTML-код страницы
        page_source = driver.page_source

    finally:
        driver.quit()

    # Парсинг HTML
    soup = BeautifulSoup(page_source, 'html.parser')

    # Если текущий снапшот ещё не содержит столбец 'volume 24(h)', то число столбцов - 10, иначе - 11
    if current_snapshot_number >= first_snapshot_with_volume:
        columns_number = 11
    else:
        columns_number = 10

    # Извлечение заголовков таблицы (почему-то нужно брать только первые 10 заголовков, далее идут повторы до 22 заголовков)
    headers = [th.text.strip() for th in soup.find_all('th')][:columns_number]

    # Извлечение данных из строк таблицы
    rows = []
    for row in soup.find_all('tr')[1:21]:  # Пропускаем первую строку с заголовками
        cells = row.find_all('td')
        if len(cells) > 0:
            row_data = [cell.text.strip() for cell in cells]
            rows.append(row_data)
    # Создание DataFrame
    temp_df = pd.DataFrame(rows, columns=headers)

    # Если текущий снапшот ещё не содержит столбец 'volume 24(h)', то добавим его в итоговый датафрейм и заполним нулями
    if current_snapshot_number < first_snapshot_with_volume:
        temp_df.insert(6,'volume (24h)', 0)

    # Сохраняем файл
    json_filename = f"{current_snapshot_date_string}.json"
    temp_df.to_json(json_filename, orient='records', lines=True, force_ascii=False)
    pause_time = random.uniform(2, 25) # 2-25 секунд между запросами
    print(f"{str(datetime.now()).split('.')[0]} Обаботан снэпшот № {current_snapshot_number + 1}, делаю паузу {round(pause_time)} сек.")
    time.sleep(pause_time)  # 2-25 секунд между запросами

2025-02-12 01:30:15 Обаботан снэпшот № 35, делаю паузу 21 сек.
2025-02-12 01:30:54 Обаботан снэпшот № 36, делаю паузу 7 сек.


С учетом случайных пауз от 2 до 25 секунд после обработки каждого снэпшота, скрипт выполнялся порядка одной минуты. Неплохо. Посмотрим на наши JSON-файлы:

In [92]:
print("Первый файл (20131222.json):")
!head -n 3 20131222.json
print("\nВторой файл (20131229.json):")
!head -n 3 20131229.json

Первый файл (20131222.json):
{"Rank":"1","Name":"BTCBitcoin","Symbol":"BTC","Market Cap":"$7,505,757,333.43","Price":"$617.18","Circulating Supply":"12,161,375 BTC","volume (24h)":0,"% 1h":"-0.68%","% 24h":"2.42%","% 7d":"-29.26%","":""}
{"Rank":"2","Name":"LTCLitecoin","Symbol":"LTC","Market Cap":"$407,576,814.52","Price":"$16.86","Circulating Supply":"24,181,092 LTC","volume (24h)":0,"% 1h":"-0.21%","% 24h":"0.65%","% 7d":"-45.92%","":""}
{"Rank":"3","Name":"XRPXRP","Symbol":"XRP","Market Cap":"$169,773,560.84","Price":"$0.02172","Circulating Supply":"7,817,889,792 XRP","volume (24h)":0,"% 1h":"-0.88%","% 24h":"0.61%","% 7d":"-25.01%","":""}



Второй файл (20131229.json):
{"Rank":"1","Name":"BTCBitcoin","Symbol":"BTC","Market Cap":"$9,082,103,472.45","Price":"$745.05","Circulating Supply":"12,189,925 BTC","volume (24h)":"$19,011,344.00","% 1h":"-0.10%","% 24h":"1.96%","% 7d":"17.84%","":""}
{"Rank":"2","Name":"LTCLitecoin","Symbol":"LTC","Market Cap":"$586,899,831.49","Price":"$24.07","Circulating Supply":"24,387,992 LTC","volume (24h)":"$11,384,235.00","% 1h":"-0.94%","% 24h":"6.35%","% 7d":"39.97%","":""}
{"Rank":"3","Name":"XRPXRP","Symbol":"XRP","Market Cap":"$212,783,895.27","Price":"$0.02722","Circulating Supply":"7,817,889,792 XRP","volume (24h)":"$56,771.56","% 1h":"-0.27%","% 24h":"-0.33%","% 7d":"22.27%","":""}


В принципе всё нормально, оба файла содержат столбец "volume (24h)", но у первого он содержит нули, а у второго - отличные от нуля значения.

### 3.7. Пакетное считывание данных

Ну что ж, настало время пройтись по всем снэпшотам сразу. С учетом того, что их ожидается 613 штук, каждый снэпшот будем класть в отдельный файл в специально отведенной папке "all_data". В случае внезапной ошибки, мы не потеряем те снэпшоты, которые уже удалось обработать, и можно будет затем продолжить процесс с последнего, а также локализовать возможный источник ошибки, если он находится в самих данных.

С учетом того, что на обработку двух снэпшотов у нас уходила минута, то на 600 уйдет примерно 300 минут или 5 часов. Запустим скриптик на ночь, а утром посмотрим, что получилось...

In [139]:
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import random
import os
import json

# Строка с датой исторически первого доступного снэпшота
first_snapshot_date_string = "20130428"
# Строка с датой исторически первого доступного снэпшота, где начали указывать колонку Volume
first_snapshot_with_volume = 35

# Номер начального снэпшота для записи в JSON-файлы
start_snapshot_number = 34
# Номер конечного снэпшота для записи в JSON-файлы
stop_snapshot_number = 35

for current_snapshot_number in range(start_snapshot_number, stop_snapshot_number + 1):
    # Указываем актуальный путь к chromedriver
    service = ChromeService(executable_path='/usr/bin/chromedriver')
    options = Options()
    options.headless = True  # Запуск в фоновом режиме

    try:
        # Переход на страницу
        driver = webdriver.Chrome(service=service, options=options)
        current_snapshot_date_string = str(pd.to_datetime(first_snapshot_date_string) + timedelta(weeks=current_snapshot_number)).split(' ')[0].replace("-","")
        driver.get("https://coinmarketcap.com/historical/" + current_snapshot_date_string +"/")
        time.sleep(15)  # Дополнительная задержка для полной загрузки данных

        # Получаем HTML-код страницы
        page_source = driver.page_source

    finally:
        driver.quit()

    # Парсинг HTML
    soup = BeautifulSoup(page_source, 'html.parser')

    # Если текущий снапшот ещё не содержит столбец 'volume 24(h)', то число столбцов - 10, иначе - 11
    if current_snapshot_number >= first_snapshot_with_volume:
        columns_number = 11
    else:
        columns_number = 10

    # Извлечение заголовков таблицы (почему-то нужно брать только первые 10 заголовков, далее идут повторы до 22 заголовков)
    headers = [th.text.strip() for th in soup.find_all('th')][:columns_number]

    # Извлечение данных из строк таблицы
    rows = []
    for row in soup.find_all('tr')[1:21]:  # Пропускаем первую строку с заголовками и берём суммарно 10 строк
        cells = row.find_all('td')
        if len(cells) > 0:
            row_data = [cell.text.strip() for cell in cells]
            rows.append(row_data)
    # Создание DataFrame
    df = pd.DataFrame(rows, columns=headers)

    # Если текущий снапшот ещё не содержит столбец 'volume 24(h)', то добавим его в итоговый датафрейм и заполним нулями
    if current_snapshot_number < first_snapshot_with_volume:
        df.insert(6,'volume (24h)', 0)

    # Сохраняем файл
    json_filename = f"all_data/{current_snapshot_date_string}.json"
    df.to_json(json_filename, orient='records', lines=True, force_ascii=False)
    pause_time = random.uniform(2, 25) # 2-25 секунд между запросами
    print(f"{str(datetime.now()).split('.')[0]} Обаботан снэпшот № {current_snapshot_number + 1}, делаю паузу {round(pause_time)} сек.")
    time.sleep(pause_time)  # 2-25 секунд между запросами

2025-01-26 22:42:44 Обаботан снэпшот № 38, делаю паузу 8 сек.
2025-01-26 22:43:12 Обаботан снэпшот № 39, делаю паузу 10 сек.
2025-01-26 22:43:48 Обаботан снэпшот № 40, делаю паузу 19 сек.
2025-01-26 22:44:29 Обаботан снэпшот № 41, делаю паузу 17 сек.
2025-01-26 22:45:07 Обаботан снэпшот № 42, делаю паузу 19 сек.
2025-01-26 22:45:45 Обаботан снэпшот № 43, делаю паузу 7 сек.
2025-01-26 22:46:13 Обаботан снэпшот № 44, делаю паузу 4 сек.
2025-01-26 22:46:42 Обаботан снэпшот № 45, делаю паузу 21 сек.
2025-01-26 22:47:27 Обаботан снэпшот № 46, делаю паузу 22 сек.
2025-01-26 22:48:13 Обаботан снэпшот № 47, делаю паузу 5 сек.
2025-01-26 22:48:39 Обаботан снэпшот № 48, делаю паузу 12 сек.
2025-01-26 22:49:11 Обаботан снэпшот № 49, делаю паузу 12 сек.
2025-01-26 22:49:41 Обаботан снэпшот № 50, делаю паузу 23 сек.
2025-01-26 22:50:23 Обаботан снэпшот № 51, делаю паузу 21 сек.
2025-01-26 22:51:04 Обаботан снэпшот № 52, делаю паузу 22 сек.
2025-01-26 22:51:50 Обаботан снэпшот № 53, делаю паузу 12 с

Судя по результатам, скрипт отработал корректно. Посмотрим в папку "all_data"

In [95]:
!ls all_data/

20130428.json  20150906.json  20180114.json  20200524.json  20221002.json
20130505.json  20150913.json  20180121.json  20200531.json  20221009.json
20130512.json  20150920.json  20180128.json  20200607.json  20221016.json
20130519.json  20150927.json  20180204.json  20200614.json  20221023.json
20130526.json  20151004.json  20180211.json  20200621.json  20221030.json
20130602.json  20151011.json  20180218.json  20200628.json  20221106.json
20130609.json  20151018.json  20180225.json  20200705.json  20221113.json
20130616.json  20151025.json  20180304.json  20200712.json  20221120.json
20130623.json  20151101.json  20180311.json  20200719.json  20221127.json
20130630.json  20151108.json  20180318.json  20200726.json  20221204.json
20130707.json  20151115.json  20180325.json  20200802.json  20221211.json
20130714.json  20151122.json  20180401.json  20200809.json  20221218.json
20130721.json  20151129.json  20180408.json  20200816.json  20221225.json
20130728.json  20151206.json  20180415

Как видно, все файлы на месте. Пора объединить их в один.

### 3.8. Объединение снэпшотов в один DataFrame

Объединим имеющиеся у нас 613 JSON-файлов с данными из снэпшотов в один общий JSON-файл с говорящим названием all_data.json, а параллельно заполним итоговый DataFrame данными.

In [111]:
# Импорт библиотек
import os
import pandas as pd
import json

df = pd.DataFrame()
temp_df = pd.DataFrame()

path_to_files = os.getcwd() + "/all_data"
for path, dirs, files in os.walk(path_to_files,):
    files.sort()
    for index, file in enumerate(files):
        if file == "all_data.json": continue
        temp_df = pd.read_json(path_to_files + "/" + file, orient='records', lines=True)    
        temp_df = temp_df.assign(date=pd.to_datetime(file.split(".")[0], format="%Y%m%d"))
        df = pd.concat([df,temp_df])
        #print(f"Обработан файл: {file}")
df.to_json(path_to_files + "/" + "all_data.json", orient='records', lines=True, force_ascii=False)
print(f"Объединение файлов завершено. Итоговый датафрейм df наполнен данными.")

Объединение файлов завершено. Итоговый датафрейм df наполнен данными.


Весь процесс занял всего 6 секунд. Отлично, посмотрим теперь на итоговый датафрейм:

In [112]:
df

,Rank,Name,Symbol,Market Cap,Price,Circulating Supply,volume (24h),% 1h,% 24h,% 7d,,date
0,1,BTCBitcoin,BTC,"$1,488,566,971.96",$134.21,"11,091,325 BTC",0,0.64%,--,--,,2013-04-28
1,2,LTCLitecoin,LTC,"$74,637,021.57",$4.3484,"17,164,230 LTC",0,0.80%,--,--,,2013-04-28
2,3,PPCPeercoin,PPC,"$7,250,186.65",$0.3865,"18,757,362 PPC",0,-0.93%,--,--,,2013-04-28
3,4,NMCNamecoin,NMC,"$5,995,997.19",$1.1072,"5,415,300 NMC",0,-0.05%,--,--,,2013-04-28
4,5,TRCTerracoin,TRC,"$1,503,099.40",$0.6469,"2,323,570 TRC",0,0.61%,--,--,,2013-04-28
...,...,...,...,...,...,...,...,...,...,...,...,...
13,14,XLMStellar,XLM,"$13,186,102,490.32",$0.4329,"30,460,651,992 XLM *","$1,178,973,781.49",-1.48%,-11.83%,2.44%,,2025-01-19
14,15,HBARHedera,HBAR,"$12,483,888,747.11",$0.3262,"38,266,149,211 HBAR *","$1,267,206,516.94",-1.48%,-8.12%,17.36%,,2025-01-19
15,16,TONToncoin,TON,"$12,063,610,686.09",$4.8533,"2,485,639,657 TON *","$362,156,523.21",-1.06%,-8.19%,-8.47%,,2025-01-19
16,17,SHIBShiba Inu,SHIB,"$11,945,149,554.24",$0.00002027,"589,255,174,736,505 SHIB *","$1,405,868,727.07",-1.07%,-10.86%,-6.00%,,2025-01-19


Выглядит солидно, но c данными явно необходимо ещё поработать:
- убрать из названий колонок спецсимволы и пробелы
- Привести в адекватный вид значения в колонке Name
- Убрать лишние знаки долларов, звездочек, и тикеров из колонки Circulating Supply


In [118]:
#Переименовываем столбцы
df = df.rename(columns={'Market Cap':'Market_Cap', 'Circulating Supply': 'Circulating_Supply', 'volume (24h)':'Volume_24h', \
    "% 1h":'1h', "% 24h":"24h", "% 7d":'7d'})
df

,Rank,Name,Symbol,Market_Cap,Price,Circulating_Supply,Volume_24h,1h,24h,7d,,date
0,1,BTCBitcoin,BTC,"$1,488,566,971.96",$134.21,"11,091,325 BTC",0,0.64%,--,--,,2013-04-28
1,2,LTCLitecoin,LTC,"$74,637,021.57",$4.3484,"17,164,230 LTC",0,0.80%,--,--,,2013-04-28
2,3,PPCPeercoin,PPC,"$7,250,186.65",$0.3865,"18,757,362 PPC",0,-0.93%,--,--,,2013-04-28
3,4,NMCNamecoin,NMC,"$5,995,997.19",$1.1072,"5,415,300 NMC",0,-0.05%,--,--,,2013-04-28
4,5,TRCTerracoin,TRC,"$1,503,099.40",$0.6469,"2,323,570 TRC",0,0.61%,--,--,,2013-04-28
...,...,...,...,...,...,...,...,...,...,...,...,...
13,14,XLMStellar,XLM,"$13,186,102,490.32",$0.4329,"30,460,651,992 XLM *","$1,178,973,781.49",-1.48%,-11.83%,2.44%,,2025-01-19
14,15,HBARHedera,HBAR,"$12,483,888,747.11",$0.3262,"38,266,149,211 HBAR *","$1,267,206,516.94",-1.48%,-8.12%,17.36%,,2025-01-19
15,16,TONToncoin,TON,"$12,063,610,686.09",$4.8533,"2,485,639,657 TON *","$362,156,523.21",-1.06%,-8.19%,-8.47%,,2025-01-19
16,17,SHIBShiba Inu,SHIB,"$11,945,149,554.24",$0.00002027,"589,255,174,736,505 SHIB *","$1,405,868,727.07",-1.07%,-10.86%,-6.00%,,2025-01-19
